In [2]:
# 查看目录
! ls /cos_person/data

data.zip
round1_iflyad_anticheat_testdata_feature.txt
round1_iflyad_anticheat_traindata.txt
sample.csv


In [137]:
! pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
    100% |################################| 1.2MB 3.0MB/s 
You are using pip version 18.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# 讯飞移动广告反欺诈数据分析

In [143]:
# 导入相关包
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import gc

pd.set_option('display.float_format',lambda x : '%.2f' % x)
plt.style.use('seaborn-dark') 
plt.rcParams['axes.unicode_minus']=False 
plt.rcParams['figure.figsize'] = (10.0, 5.0)
plt.rcParams['font.sans-serif'] = ['SimHei']

In [154]:
path = '/cos_person/data/'
train = pd.read_csv(path+'round1_iflyad_anticheat_traindata.txt', delimiter='\t')
test = pd.read_csv(path+'round1_iflyad_anticheat_testdata_feature.txt', delimiter='\t')
# 合并train和test文件
df = pd.concat([train.assign(is_train=1), test.assign(is_train=0)])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [22]:
df.head()

,adidmd5,adunitshowid,apptype,carrier,city,dvctype,h,idfamd5,imeimd5,ip,...,orientation,os,osv,pkgname,ppi,province,reqrealip,sid,ver,w
0,eb4fa0cf77cdd57a6993a4e42c8fa4cf,907d0f8c29663840491577a21c7b612a,280.00,46000.00,石家庄市,2.00,2340.00,empty,930b3a8ecff4f586c63fec5b96693f32,183.197.47.83,...,0.00,Android,8.1.0,2d2664e827bcbb8b07100c7fbe072e9b,3.00,-1.00,180.76.181.7,d7460126-e071-4979-9ee8-42f72777a28a-156009070...,11.11.0,1080.00
1,a4ae6efcfaee62fb6da44a9c05753259,10199dc8fea2e173525bc3151bd84312,319.00,0.00,开封市,2.00,1080.00,empty,6e38deaa1f7b4118015c550bb21913d2,106.34.14.149,...,0.00,android,Android_9,empty,0.00,-1.00,47.95.111.138,b660d559-db97-4b5f-9bd2-2450cb89ce77-156005074...,NaN,2040.00
2,3404f799628be2cf8fda7428aac2cca4,83f2ecfe65f936f5f2ed59f8e8ff1d01,273.00,46000.00,长春市,2.00,2196.00,empty,3fa42b06332c729f530b973973fabe3d,223.104.16.151,...,1.00,Android,8.1.0,16b81f93f4b1a35cebbf15f07683f171,0.00,-1.00,39.96.159.155,f49a740e-66c3-4605-9b67-4d3079fe69cb-156008914...,3.2.1.0524.1958,1080.00
3,941517d46b7261d98592425672bb86f5,9f1eadd9092b19bc86ee0cacde1c867f,265.00,0.00,曲靖市,2.00,0.00,empty,04420edf96dde14c357db3e83d0b2310,223.104.239.101,...,0.00,android,7.0,empty,0.00,-1.00,47.101.65.230,fd60d096-f168-4540-b782-729d64d0fcc6-156006253...,4.7.5,0.00
4,fc08a6724db4c2dcf9b5af3a4e0b6f38,2af944462e43cd2f59acbbfd37445413,336.00,46000.00,深圳市,2.00,2244.00,empty,23e94b65bab381bf57169149a20c0520,220.152.155.170,...,0.00,Android,9,cf4821986014a7fef9d7b7ad8de655e4,0.00,-1.00,39.96.183.23,a037b032-a5c7-40ea-9161-26b118b12406-156007938...,228,1080.00


数据类别	变量	数值格式	解释
基本数据	sid	string	样本id/请求会话sid
基本数据	label	int	是否作弊
媒体信息	pkgname	string	包名(MD5加密)
媒体信息	ver	string	app版本
媒体信息	adunitshowid	string	对外广告位ID（MD5加密）
媒体信息	mediashowid	string	对外媒体ID（MD5加密）
媒体信息	apptype	int	app所属分类
时间	nginxtime	bigint	请求到达服务时间，单位ms
IP信息	ip	string	客户端IP地址
IP信息	city	string	城市
IP信息	province	string	省份
IP信息	reqrealip	string	请求的http协议头携带IP，有可能是下游服务器的ip
设备信息	adidmd5	string	Adroid ID的MD5值
设备信息	imeimd5	string	imei的MD5值
设备信息	idfamd5	string	idfa的MD5值
设备信息	openudidmd5	string	openudid的MD5值
设备信息	macmd5	string	mac的MD5值
设备信息	dvctype	int	设备类型 0 – 未知,1 – PC,2 – 手机, 3– 平板,4– 电视盒,5– 智能电视,6 – 可穿戴设备,7 – 智能家电,8 - 音箱,9 - 智能硬件
设备信息	model	string	机型
设备信息	make	string	厂商
设备信息	ntt	int	网络类型 0-未知, 1-有线网, 2-WIFI, 3-蜂窝网络未知, 4-2G, 5-3G, 6–4G
设备信息	carrier	string	运营商 0-未知, 46000-移动, 46001-联通, 46003-电信
设备信息	os	string	操作系统 : Android, iOS
设备信息	osv	string	操作系统版本
设备信息	orientation	int	横竖屏:0竖屏，1横屏
设备信息	lan	string	语言
设备信息	h	int	设备高
设备信息	w	int	设备宽
设备信息	ppi	int	屏幕密度

In [38]:
""" 数据属性 """
df.info()

print()
print("共有数据集：", df.shape[0])
print("共有测试集：", test.shape[0])
print("共有训练集：", train.shape[0])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100000 entries, 0 to 99999
Data columns (total 30 columns):
adidmd5         1100000 non-null object
adunitshowid    1100000 non-null object
apptype         1100000 non-null float64
carrier         1100000 non-null float64
city            1083340 non-null object
dvctype         1100000 non-null float64
h               1100000 non-null float64
idfamd5         1100000 non-null object
imeimd5         1100000 non-null object
ip              1100000 non-null object
is_train        1100000 non-null int64
label           1000000 non-null float64
lan             699138 non-null object
macmd5          1100000 non-null object
make            964657 non-null object
mediashowid     1100000 non-null object
model           1095438 non-null object
nginxtime       1100000 non-null float64
ntt             1100000 non-null float64
openudidmd5     1100000 non-null object
orientation     1100000 non-null float64
os              1100000 non-null object
osv 

In [40]:
""" 数据类别 """
for i,name in enumerate(df.columns):
    name_sum = df[name].value_counts().shape[0]
    print("{:2}、{:15}      The number of types of features is：{}".format(i + 1, name, name_sum))

 1、adidmd5              The number of types of features is：780369
 2、adunitshowid         The number of types of features is：800
 3、apptype              The number of types of features is：91
 4、carrier              The number of types of features is：5
 5、city                 The number of types of features is：331
 6、dvctype              The number of types of features is：3
 7、h                    The number of types of features is：985
 8、idfamd5              The number of types of features is：360
 9、imeimd5              The number of types of features is：1021836
10、ip                   The number of types of features is：813719
11、is_train             The number of types of features is：2
12、label                The number of types of features is：2
13、lan                  The number of types of features is：33
14、macmd5               The number of types of features is：329184
15、make                 The number of types of features is：2727
16、mediashowid          The number of types of feat

In [15]:
train.isnull().sum()

sid                  0
label                0
pkgname              0
ver             321824
adunitshowid         0
mediashowid          0
apptype              0
nginxtime            0
ip                   0
city             15193
province             0
reqrealip            0
adidmd5              0
imeimd5              0
idfamd5              0
openudidmd5          0
macmd5               0
dvctype              0
model             4467
make            123839
ntt                  0
carrier              0
os                   0
osv              12997
orientation          0
lan             367527
h                    0
w                    0
ppi                  0
dtype: int64

In [42]:
""" 数据统计 """
df.describe()

,apptype,carrier,dvctype,h,is_train,label,nginxtime,ntt,orientation,ppi,province,w
count,1100000.00,1100000.00,1100000.00,1100000.00,1100000.00,1000000.00,1100000.00,1100000.00,1100000.00,1100000.00,1100000.00,1100000.00
mean,295.94,40019.02,1.64,1269.68,0.91,0.48,1559844776087.79,3.10,0.28,72.16,0.44,705.23
std,42.94,15471.53,0.81,852.58,0.29,0.50,186850053.67,1.85,1.05,167.83,6.13,504.54
min,-1.00,-1.00,0.00,-1.00,0.00,0.00,1559491202247.00,0.00,0.00,0.00,-1.00,-1.00
25%,273.00,46000.00,2.00,720.00,1.00,0.00,1559691357899.50,2.00,0.00,0.00,-1.00,360.00
50%,301.00,46000.00,2.00,1280.00,1.00,0.00,1559836800449.00,2.00,0.00,0.00,-1.00,720.00
75%,326.00,46000.00,2.00,2048.00,1.00,1.00,1559998804440.75,5.00,1.00,0.00,-1.00,1080.00
max,381.00,46003.00,3.00,9024.00,1.00,1.00,1560182396964.00,7.00,90.00,1000.00,34.00,9024.00


In [155]:
# 缺失值预处理函数
def preprocessing(df_):
    df = df_.copy()
    
    df['nginxdate'] = pd.to_datetime(df['nginxtime']* 1e+6)
    df['date'] = df['nginxdate'].dt.date
    df['day'] = df['nginxdate'].dt.day
    df['hour'] = df['nginxdate'].dt.hour
    df['dayofweek'] = df['nginxdate'].dt.dayofweek
      
    
    df['ver'] =  df['ver'].fillna(str(-1)) # 媒体app版本
    # 缺失城市填充 后面可以考虑该问题的优化
    df['city'] = df.groupby('province')['city'].transform(lambda x:x.fillna(method = 'ffill'))
    df['city'] = df['city'].fillna(str(-1))
    df['model'] = df['model'].fillna(str(-1))
    df['make'] = df['make'].fillna(str(-1))
    df['osv'] = df['osv'].fillna(df['osv'].mode()[0])
    begin_time= df['sid'].apply(lambda x:int(x.split('-')[-1])) ##请求会话时间
    df['usetime'] = df['nginxtime'] - begin_time ##请求会话时间 与 请求到达服务时间的差
    
    
    df['lan'] =  df['lan'].fillna(str(-1)) # 语言
    language = ['zh-CN','Zh-CN', 'zh-cn', 'cn', 'zh_CN', 'zh', 'ZH', 'CN', 'tw','zh_CN_#Hans','zh-TW','zh-HK','zh-US','zh_HK_#Hant',
           'zh-MO','zh-','zh_TW']
    language2 = ['zh-CN','Zh-CN', 'zh-cn', 'cn', 'zh_CN', 'zh', 'ZH', 'CN', 'tw','zh_CN_#Hans','zh-TW','zh-HK','zh-US','zh_HK_#Hant',
           'zh-MO','zh-','zh_TW', '-1']
    df.loc[df['lan'].isin(language),'lan'] = 'cn'
    df.loc[~ df['lan'].isin(language2),'lan'] = 'other'
    
    del df['nginxtime']
    del df['nginxdate']
    del df['sid']
    del df['province']
    return df

In [156]:
# 创建测试集 ，为了防止采样偏差，这里使用分层采样
# 测试集通常被忽略，但实际是机器学习非常重要的一部分。
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=2018)
for train_index, test_index in split.split(train, train["label"]):
    strat_train_set=train.loc[train_index]
    strat_test_set=train.loc[test_index]

# 分层采样就是按照这里给的 status 分类比例来的
train["label"].value_counts()/len(train)

xx = strat_train_set.copy()
X_train= xx.drop(["label"], axis=1)
y_train= xx[["label"]]
xx = strat_train_set.copy()
X_test= xx.drop(["label"], axis=1)
y_test= xx[["label"]]

In [157]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700000 entries, 286924 to 747476
Data columns (total 28 columns):
sid             700000 non-null object
pkgname         700000 non-null object
ver             475038 non-null object
adunitshowid    700000 non-null object
mediashowid     700000 non-null object
apptype         700000 non-null float64
nginxtime       700000 non-null float64
ip              700000 non-null object
city            689382 non-null object
province        700000 non-null float64
reqrealip       700000 non-null object
adidmd5         700000 non-null object
imeimd5         700000 non-null object
idfamd5         700000 non-null object
openudidmd5     700000 non-null object
macmd5          700000 non-null object
dvctype         700000 non-null float64
model           696892 non-null object
make            613348 non-null object
ntt             700000 non-null float64
carrier         700000 non-null float64
os              700000 non-null object
osv             6909

In [158]:
# 完整预处理流水线


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


# 类别属性
cat_columns = ['pkgname', 'ver', 'adunitshowid', 'mediashowid', 'apptype', 'ip', 'city', 'reqrealip', 'adidmd5',
              'idfamd5', 'openudidmd5', 'macmd5', 'dvctype', 'model', 'make', 'ntt', 'carrier', 'os', 'osv', 'orientation',
              'lan', 'date', 'day', 'hour', 'dayofweek', 'imeimd5']

def transform(df):
    """ 
        特征转换
    """

    # 数值属性
#     num_cols = X_train.columns
#     steps = [('ss', StandardScaler())]
#     num_pipe = Pipeline(steps)
#     transformers = [
#         ('num', num_pipe, num_cols),
#         ("cat", LabelEncoder(), cat_columns),
#     ]
#     ct = ColumnTransformer(transformers=transformers)
#     X_num_transformed = ct.fit_transform(df)
    
    
    labelencoder_X = LabelEncoder()
    for item in cat_columns:
        df[item] = labelencoder_X.fit_transform(df[item])
        df[item] = df[item]
    
    return df


X_train = preprocessing(X_train)
X_test = preprocessing(X_test)
X_train = transform(X_train)
X_test = transform(X_test)

In [162]:
X_test.head()

,pkgname,ver,adunitshowid,mediashowid,apptype,ip,city,reqrealip,adidmd5,imeimd5,...,orientation,lan,h,w,ppi,date,day,hour,dayofweek,usetime
286924,1996,2123,532,31,62,288980,0,255,342684,247338,...,0,1,2220.00,1080.00,0.00,5,5,23,4,-2.00
203572,29,1,590,6,68,247264,286,2478,470444,480766,...,0,0,0.00,0.00,0.00,6,6,7,5,1.00
400517,152,2155,562,182,58,451359,181,4256,108109,286625,...,1,1,2028.00,1080.00,480.00,5,5,4,4,0.00
702825,92,1,282,276,12,247886,68,3731,162047,264077,...,0,0,1920.00,1080.00,0.00,3,3,9,2,1.00
454481,210,1591,194,240,48,378546,79,5745,470444,348993,...,1,1,748.00,360.00,0.00,3,3,23,2,2.00


In [160]:
test = pd.read_csv(path+'round1_iflyad_anticheat_testdata_feature.txt', delimiter='\t')
test_sid = test['sid']
test_set = preprocessing(test)
test_set = transform(test_set)

In [163]:
# 模型训练
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import KFold, StratifiedKFold

result_pd = pd.DataFrame(data=test_sid, columns=['sid'])
result_pd['label'] = 0

n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.04, random_state=42
    )


for train_idx, val_idx in kfold.split(X_train):
    clf.random_state = clf.random_state + 1
    
    train_x1 = X_train.iloc[train_idx]
    train_y1 = y_train.iloc[train_idx]
    
    test_x1 = X_train.iloc[val_idx]
    test_y1 = y_train.iloc[val_idx]
    
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)], eval_metric='accuracy',early_stopping_rounds=20,
           )
    result_pd['label'] += clf.predict_proba(test_set)[:,1]
    

y_pred=clf.predict(X_test)
y_predprob=clf.predict_proba(X_test)[:,1]
print("AUC Score(Train data): %f" % roc_auc_score(y_test,y_predprob))


[1]	training's auc: 0.932295	valid_1's auc: 0.931363
Training until validation scores don't improve for 20 rounds.
[2]	training's auc: 0.943194	valid_1's auc: 0.942657
[3]	training's auc: 0.945598	valid_1's auc: 0.945351
[4]	training's auc: 0.94553	valid_1's auc: 0.945092
[5]	training's auc: 0.94738	valid_1's auc: 0.946834
[6]	training's auc: 0.948805	valid_1's auc: 0.948252
[7]	training's auc: 0.947513	valid_1's auc: 0.946909
[8]	training's auc: 0.949452	valid_1's auc: 0.948913
[9]	training's auc: 0.950483	valid_1's auc: 0.949977
[10]	training's auc: 0.950709	valid_1's auc: 0.950293
[11]	training's auc: 0.952009	valid_1's auc: 0.951622
[12]	training's auc: 0.952937	valid_1's auc: 0.952483
[13]	training's auc: 0.953341	valid_1's auc: 0.952915
[14]	training's auc: 0.953557	valid_1's auc: 0.953168
[15]	training's auc: 0.953299	valid_1's auc: 0.952912
[16]	training's auc: 0.953516	valid_1's auc: 0.953182
[17]	training's auc: 0.954102	valid_1's auc: 0.953799
[18]	training's auc: 0.954586	v

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [164]:
print("f1 Score(Train data): %f" % f1_score(y_test,y_pred))

f1 Score(Train data): 0.935649


In [165]:
# 保存

y_pred=clf.predict(X_test)
result_pd['label'] = result_pd['label']/n_splits
result_pd.to_csv('submit_result.csv', index=False)